In [1]:
from __future__ import print_function
from keras.models import Sequential, Graph
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, GRU
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Nadam, RMSprop
from keras.utils.data_utils import get_file
from datetime import datetime
import numpy as np
import rethinkdb as r
import random
import sys

Using TensorFlow backend.


In [3]:
# Open dataset
text = open('../data/makeuptutorial.txt').read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

corpus length: 2814918
total chars: 59
nb sequences: 938293
Vectorization...
